In [4]:
import pymysql
import requests
import json
import pandas as pd
from datetime import datetime
from time import sleep
import sys

In [5]:
# MySQL Connection 연결 (DB 연결)
def setting_db():
    conn = pymysql.connect(host="localhost", 
                       port=3306, user="root", 
                       passwd="qlqtmajrwk",
                       db="yejinyDB",
                       charset="utf8")
    cursor = conn.cursor(pymysql.cursors.DictCursor) # 리스트로 받기(tuple은 conn.cursor())
    return conn, cursor

In [3]:
# Rcp table 만들기
conn, cursor = setting_db()
# 공시접수번호 rcp_no 종목코드 crp_cd 공시접수일자시각 rcp_dt
# 현재주가 now_st 현재전일비 now_diff 현재거래량 volume 현재거래대금 now_trd
# 공시카테고리 rpt_cat 공시내용 rpt_nm 공시링크 rpt_link 참고사항 rmk
sql = '''CREATE TABLE Rcp (rcp_no nvarchar(45) NOT NULL,
                            crp_cd nvarchar(45) NOT NULL,
                            rcp_dt datetime NOT NULL,
                            now_st bigint,
                            now_diff bigint,
                            now_volume bigint,
                            now_trd bigint,
                            rpt_cat nvarchar(45),
                            rpt_nm text NOT NULL,
                            rpt_link nvarchar(100) NOT NULL,
                            rmk nvarchar(45),                            
                            PRIMARY KEY(rcp_no)
                            );
                            '''
cursor.execute(sql)
conn.commit()
conn.close()

InternalError: (1050, "Table 'Rcp' already exists")

In [6]:
def get(crp_cd):
    URL = 'http://polling.finance.naver.com/api/realtime.nhn?query=SERVICE_ITEM:'+crp_cd
    data = {'outer': {'inner': 'value'}}
    res = requests.post(URL, data=json.dumps(data))
    tmp = res.json()['result']['areas'][0]['datas'][0]
    return tmp

In [7]:
get('000720')

{'aa': 35741512200,
 'aq': 598279,
 'bps': 58029.41091,
 'cd': '000720',
 'cnsEps': 4748,
 'cr': 2.05,
 'cv': 1200,
 'dv': 500.0,
 'eps': 1810,
 'hv': 60100,
 'keps': 1810,
 'll': 41000,
 'lv': 59000,
 'ms': 'OPEN',
 'mt': '1',
 'nav': None,
 'nm': '현대건설',
 'nv': 59700,
 'ov': 59200,
 'pcv': 58500,
 'rf': '2',
 'sv': 58500,
 'tyn': 'N',
 'ul': 76000}

In [8]:
def get_now(crp_cd):
    URL = 'http://polling.finance.naver.com/api/realtime.nhn?query=SERVICE_ITEM:'+crp_cd
    data = {'outer': {'inner': 'value'}}
    res = requests.post(URL, data=json.dumps(data))
    tmp = res.json()['result']['areas'][0]['datas'][0]
    result = {'now_st':tmp['nv'], 'now_diff':tmp['nv']-tmp['sv'], 'now_volume':tmp['aq'], 'now_trd':tmp['aa']}
    return result

In [9]:
get_now('005380')

{'now_diff': -1000,
 'now_st': 124500,
 'now_trd': 12134358500,
 'now_volume': 97500}

In [10]:
log_list = []

In [ ]:
while True:
    try:
        today = datetime.today().strftime('%Y-%m-%d')
        conn, cursor = setting_db()
        sql = "select rcp_no from Rcp where date(rcp_dt) = '"+today+"'"
        cursor.execute(sql)
        old = cursor.fetchall()
        conn.commit()
        conn.close()

        old_no = []
        for i in old[0:(len(old))]:
            rcp_no = i['rcp_no']
            old_no.append(rcp_no)        

        url_dart_list='http://dart.fss.or.kr/api/search.json?auth=92613a8cac3325e99a4b684b864aee1fd24eb775' #yejin's
        new = requests.get(url_dart_list).json()['list']

        new_no = []
        for i in new[0:(len(new))]:
            rcp_no = i['rcp_no']
            new_no.append(rcp_no)        

        change_no = set(new_no).difference(old_no)
        if change_no:
            print('detect')
            change = []
            for i in change_no:
                for li in new:
                    if li['rcp_no'] == i:
                        if len(li['crp_cd']) == 6: # Y, K, N
                            now_dt = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                            now_tmp = get_now(li['crp_cd'])
                            change.append((li['rcp_no'], li['crp_cd'], now_dt, now_tmp['now_st'], now_tmp['now_diff'], now_tmp['now_volume'], now_tmp['now_trd'], li['rpt_nm'], 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo='+li['rcp_no'], li['rmk']))
                        else:
                            change.append((li['rcp_no'], li['crp_cd'], now_dt, -1, 0, 0, 0, li['rpt_nm'], 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo='+li['rcp_no'], li['rmk']))

            conn, cursor = setting_db()
            sql = '''insert into Rcp(
                    rcp_no, crp_cd, rcp_dt, now_st, now_diff, now_volume, now_trd, rpt_nm, rpt_link, rmk)
                    values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'''
            cursor.executemany(sql, change)
            conn.commit()  
            conn.close()

        else:
            now = datetime.today().time()
            if now.minute % 5 == 0:
                print('nothing changed', now)
            sleep(10)

    except:
        e = sys.exc_info()
        nowtime = datetime.now()
        nowtime = nowtime.strftime('%Y-%m-%d %H:%M:%S')
        log_list.append({'time' : nowtime, 'error' : e})
        log_time = open('logfile/Rcp_log_time.txt', 'w')
        print(log_list, file = log_time)
        log_time.close()
        pass

detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect

detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect

detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect

detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect

detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect

detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect

detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect

detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect
detect